In [29]:
from util.s3_util import S3
import requests
import json
import uuid
from util.time_util import get_time_string

string_now = get_time_string()
DISCORD_TOKEN = 'NzYzNzU3ODQ0NzI1NzYwMDUw.GknV7D.hj4lWVnXgruP1hjiTaIfB2IFEzVrD4J3l9IWh0'

s3_discord = S3(bucket_name=f'discord-{string_now}')


In [ ]:
# Step 1: crawl save to s3

def crawl_message(url):
    response = requests.get(url, headers={'Authorization': DISCORD_TOKEN})
    print(response)
    if response.status_code != 200:
        return None
    
    result = response.json()
    id_before = result[-1]['id']

    s3_discord.put_object(json.dumps(result), f'{str(uuid.uuid1())}')

    return id_before

def crawl_channel(channel_id):
    url = f'https://discord.com/api/v9/channels/{channel_id}/messages?limit=50'

    id_before = crawl_message(url)

    if id_before is None:
        return

    while True:
        id_before = crawl_message(url + f'&before={id_before}')
        if id_before is None:
            break
        import time
        time.sleep(1)

# channel_id = '1008571102328541215' # newbie_127
# channel_id = '1008571159043903509' # newbie_157
channel_id = '1008571225309728878' # newbie_187
crawl_channel(channel_id)

In [54]:
# step2: get objects from s3 and clean

def clean_message(msg):
    message_id = msg.get('id', None)
    # channel_id = msg.get('channel_id', '')
    attachments = msg.get('attachments', [])
    prompt = msg.get('content', '')
    author = msg.get('author', None)
    username = author.get('username', '')

    if message_id is None \
        or len(attachments) == 0 \
            or username != 'Midjourney Bot' \
        or not prompt \
    or 'Image #' in prompt:
        return None

    if len(prompt.split('**')) < 2:
        return None

    clean_prompt = prompt.split('**')[1]

    if len(clean_prompt.split()) < 2:
        return None

    image_src = attachments[0]['url']

    return {
        'message_id': message_id,
        # 'channel_id': channel_id,
        'prompt': clean_prompt,
        'image_src': image_src
    }

def get_list_clean_message():
    result = []
    for obj in s3_discord.get_list_objects():
        body = obj.get()['Body'].read()
        list_message = json.loads(body)
        
        for message in list_message:
            _clean_message = clean_message(message)
            if _clean_message is not None:
                result.append(_clean_message)

    return result

list_clean_message = get_list_clean_message()
list_clean_message[0]

{'message_id': '1134413285979074601',
 'prompt': 'a bulldog icon faces people in the background with an emoji, in the style of playful simplicity, minimalistic, manapunk, innovative page design, disfigured forms, playful ，art of line --ar 27:25',
 'image_src': 'https://cdn.discordapp.com/attachments/1008571225309728878/1134413285391863940/michellebaker5043_a_bulldog_icon_faces_people_in_the_background_6daf4286-9fcb-4a7d-b907-17b5b0edefa8.png'}

In [59]:
# step 3: handle duplicate

MAX_DUPLICATE_PERCENTAGE = 0.3

def calculate_duplicate(sen1, sen2):
    list1 = sen1.split()
    list2 = sen2.split()

    count = 0
    for (word1, word2) in zip(list1, list2):
        if word1 == word2:
            count += 1

    duplicate_percent = count / (len(list2) + 1)
    return round(duplicate_percent, 3)

list_clean_message = get_list_clean_message()
list_unique_message = [list_clean_message[0]]

for message in list_clean_message:
    for unique_message in list_unique_message:
        sen1 = unique_message['prompt']
        sen2 = message['prompt']

        duplicate_percentage = calculate_duplicate(sen1, sen2)

        if duplicate_percentage > MAX_DUPLICATE_PERCENTAGE:
            break
    list_unique_message.append(message)

len(list_unique_message)

115